In [1]:
#!pip install transformers
#!pip install datasets
#!pip install rouge_score
#!pip install sentencepiece

In [2]:
import torch
import pandas as pd
import numpy as np
import nltk

from torch import nn
from torch.utils.data import DataLoader
from transformers import Trainer, AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, \
  DataCollatorForLanguageModeling, T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

from sklearn.metrics import mean_squared_error, precision_score
from datasets import load_dataset, Dataset, load_metric
from sklearn.model_selection import train_test_split


nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/leczhang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

In [4]:
data = pd.read_csv("review_simplified.csv")
train_df, test_df = train_test_split(data, test_size=0.2, random_state=1)

train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)

In [5]:
prefix = "Write an introduction from reviews about "

def tokenize_function(examples):
    
    inputs = []
    for i in range(len(examples["review_simple"])):
        inputs.append(prefix + examples["Name"][i] + " : " + examples["review_simple"][i])
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["Introduction"], max_length=512, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
    #return inputs


train_dataset = train_data.map(tokenize_function, batched=True)
test_dataset = test_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/2019 [00:00<?, ? examples/s]

/home/leczhang/.local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/505 [00:00<?, ? examples/s]

In [6]:
batch_size = 4

args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    #fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
)

In [7]:
data_collator = DataCollatorForSeq2Seq(tokenizer)
metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

/tmp/ipykernel_1901777/1112966091.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [8]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
model.config.max_length = 300
model.config.min_length = 100
model.config.no_repeat_ngram_size = 3

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, pos_review_content, url, review_num, pos_review_title, review_simple, Name, overall_rate, Description, Title. If __index_level_0__, pos_review_content, url, review_num, pos_review_title, review_simple, Name, overall_rate, Description, Title are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/home/leczhang/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2019
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gr

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
100,3.142200,2.906342,36.532100,10.561800,20.595900,35.230700,285.560400
200,3.112300,2.889761,37.039800,10.684200,20.591100,35.672300,279.689100
300,3.117000,2.879207,36.543200,10.393700,20.349800,35.241700,281.402000
400,3.063000,2.870202,37.292800,10.590000,20.470000,36.004900,284.772300
500,3.076200,2.866529,37.489500,10.692400,20.668300,36.172200,285.566300
600,3.029800,2.861445,36.930400,10.447100,20.422900,35.643800,280.000000
700,3.024900,2.856382,37.174200,10.537000,20.519800,35.891900,283.322800
800,3.025500,2.852888,37.460900,10.546600,20.579900,36.092400,287.429700
900,3.009400,2.850534,37.461400,10.743200,20.578000,36.174600,287.574300
1000,3.042400,2.848316,37.527900,10.751000,20.613300,36.196300,287.497000


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, pos_review_content, url, review_num, pos_review_title, review_simple, Name, overall_rate, Description, Title. If __index_level_0__, pos_review_content, url, review_num, pos_review_title, review_simple, Name, overall_rate, Description, Title are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 505
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, pos_review_content, url, review_num, pos_review_title, review_simple, Name, overall_rate, Description, Title. If __index_level_0__, pos_review_content, url, review_num, pos_review_title, review_simple, Name, overall_rate, Description, Title are not expected by `T5ForCond

***** Running Evaluation *****
  Num examples = 505
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, pos_review_content, url, review_num, pos_review_title, review_simple, Name, overall_rate, Description, Title. If __index_level_0__, pos_review_content, url, review_num, pos_review_title, review_simple, Name, overall_rate, Description, Title are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 505
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1200
Configuration saved in ./results/checkpoint-1200/config.json
Model weights saved in ./results/checkpoint-1200/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1200/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1200/special_tokens_map.json
Copy vocab file to ./resu

TrainOutput(global_step=1515, training_loss=3.041979167170257, metrics={'train_runtime': 7123.6812, 'train_samples_per_second': 0.85, 'train_steps_per_second': 0.213, 'total_flos': 4124420588381184.0, 'train_loss': 3.041979167170257, 'epoch': 3.0})